#### Busqueda tipo waze/google maps

Nuestro objetivo es crear una heuristica creativa y util para  encontrar la mejor ruta entre dos lugares del municipio de Envigado   

PASO A PASO 
1. Librerías 
2. Inicialización del grafo  
  2.1  Creación del grafo  
  2.2  Creación de los nodos
3. Manipulación del grafo  
  3.1  Velocidades y travel time  
  3.2  Superficie 
  3.3  Variables adicionales 

4. Pesos  
  4.1 Peso de superficie y función  
  4.2 Pesos de variables adicionales  
  4.3 Heurística final 

5. Enrutamiento 
  5.1 Definición Start/End  
  5.2 Rutas
  
    - Definición de rutas  
    - Gráficas 
    - Comparaciones

   5.3 Funciones complementarias  



#### 1. Librerías 

In [51]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString

import matplotlib.pyplot as plt
import plotly_express as px

import networkx as nx
import osmnx as ox

### 2 Grafo
#### 2.1 Creación del grafo 


In [52]:
G = ox.graph_from_place('Envigado, Antioquia, Colombia', network_type='drive', simplify=False)


#### 2.2 Creación Nodos y Aristas 
Con esta función graph to *graph_to_gdfs(G)* traemos los nodos y aristas de nuestro grafo como un DF

In [53]:
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)

In [54]:
gdf_edges.head()

osmid    highway lanes maxspeed  \
u         v          key                                         
309313450 414940715  0    1019189334  secondary     2       50   
          440765240  0    1019189334  secondary     2       50   
309313451 414940727  0    1019189334  secondary     2       50   
          414940715  0    1019189334  secondary     2       50   
309313454 4124260034 0    1019190824  secondary     2       60   

                                         name  oneway  reversed     length  \
u         v          key                                                     
309313450 414940715  0    Variante Las Palmas   False     False  24.138083   
          440765240  0    Variante Las Palmas   False      True  91.294214   
309313451 414940727  0    Variante Las Palmas   False     False  26.537531   
          414940715  0    Variante Las Palmas   False      True  27.314193   
309313454 4124260034 0    Variante Las Palmas   False     False  25.381809   

                          ref width junction bridge access  \
u         v          key                                     
309313450 414940715  0    NaN   NaN      NaN    NaN    NaN   
          440765240  0    NaN   NaN      NaN    NaN    NaN   
309313451 414940727  0    NaN   NaN      NaN    NaN    NaN   
          414940715  0    NaN   NaN      NaN    NaN    NaN   
309313454 4124260034 0    NaN   NaN      NaN    NaN    NaN   

                                                                   geometry  
u         v          key                                                     
309313450 414940715  0    LINESTRING (-75.51982 6.15843, -75.52004 6.15839)  
          440765240  0    LINESTRING (-75.51982 6.15843, -75.51903 6.15866)  
309313451 414940727  0     LINESTRING (-75.52028 6.1584, -75.52051 6.15847)  
          414940715  0     LINESTRING (-75.52028 6.1584, -75.52004 6.15839)  
309313454 4124260034 0     LINESTRING (-75.5215 6.15963, -75.52164 6.15981)

### 3. Manipulación del Df
Debemos tener las variables necesarias para poder hacer nuestra heuristica 

#### 3.1 Velocidades y travel time
 Vamos a llenar todas las velocidades para poder crear el travel time 


In [55]:
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)


#### 3.2 Altura Nodos
Importaremos la altura de los nodos para poder hacer el cálculo de la pendiente de las aristas 


#### 3.3 Superficie
Se importa desde features la superficie de las aristas  
Es importante resaltar la forma en que se junta gdf_surface con gdf_eges ya que en esta función se hace el manejo de los index para que el grafo se pueda actualizar de manera correcta 

In [56]:
gdf_surface = ox.features.features_from_place("Envigado, Antioquia, Colombia", tags={'surface': True})

def add_surface_data_safe(gdf_edges, gdf_surface):
    edges_with_surface = gdf_edges.copy()
    
    # Inicializar columna surface
    edges_with_surface['surface'] = 'asphalt'  # valor por defecto
    
    # Para cada feature de superficie, encontrar aristas que intersectan
    for idx, surface_feature in gdf_surface.iterrows():
        if pd.isna(surface_feature['surface']):
            continue
            
        # Encontrar aristas que intersectan con esta superficie
        mask = edges_with_surface.geometry.intersects(surface_feature.geometry)
        
        # Asignar el tipo de superficie
        edges_with_surface.loc[mask, 'surface'] = surface_feature['surface']
    
    return edges_with_surface


Verificación de los index y de cantidad de superficies 

In [57]:
gdf_edgesMod = add_surface_data_safe(gdf_edges, gdf_surface)

print("Índices originales:", gdf_edges.index.names)
print("Índices modificados:", gdf_edgesMod.index.names)
# Conteo por tipo de superficie
conteo = gdf_edgesMod['surface'].value_counts(dropna=False)
print(conteo)

Gsurface = ox.graph_from_gdfs(gdf_nodes, gdf_edgesMod)


Índices originales: ['u', 'v', 'key']
Índices modificados: ['u', 'v', 'key']
surface
asphalt            31926
unpaved             2758
paved               2075
gravel               231
concrete             151
concrete:lanes        54
paving_stones         32
cobblestone            8
concrete:plates        3
Name: count, dtype: int64


#### 3.3 Variables adicionales 
- Hospitales 
- Parques 
- Escuelas 

Se adquieren las variables necesarias con su geometría, y con este dato se adjuntan a la arista más cercana 

In [58]:
gdf_hospitals = ox.features_from_place("Envigado", tags={'amenity': 'hospital'})
gdf_schools = ox.features_from_place("Envigado", tags={'amenity': 'school'})
gdf_parks = ox.features_from_place("Envigado", tags={'leisure': 'park'})

def add_proximity_features(G, gdf_features, feature_name, buffer_distance=0.001):
    for u, v, key, data in G.edges(keys=True, data=True):
        edge_geom = data['geometry']
        buffer_zone = edge_geom.buffer(buffer_distance)
        
        # Contar features cercanos
        nearby_features = gdf_features[gdf_features.geometry.intersects(buffer_zone)]
        count = len(nearby_features)
        
        # Añadir al grafo
        G[u][v][key][feature_name] = count

add_proximity_features(Gsurface, gdf_hospitals, 'nearby_hospitals')
add_proximity_features(Gsurface, gdf_schools, 'nearby_schools')
add_proximity_features(Gsurface, gdf_parks, 'crosses_park')

Verificación de nuevas variables 

In [59]:
print("Hospitales encontrados:", len(gdf_hospitals))
print("Escuelas encontradas:", len(gdf_schools))
print("Parques encontrados:", len(gdf_parks))


# Ver si tienen geometría válida
if len(gdf_hospitals) > 0:
    print("Hospitales tienen geometría:", not gdf_hospitals.geometry.empty)
if len(gdf_schools) > 0:
    print("Escuelas tienen geometría:", not gdf_schools.geometry.empty)

Hospitales encontrados: 4
Escuelas encontradas: 94
Parques encontrados: 25
Hospitales tienen geometría: True
Escuelas tienen geometría: True


### 4 Pesos (heurística)
#### 4.1 Pesos de superficie

In [60]:
surface_weights = {
    'asphalt': 1.0,             
    'concrete': 1.1,
    'concrete:lanes': 1.1,
    'concrete:plates': 1.2,
    'paved': 1.2,
    'paving_stones': 1.4,
    'fine_gravel': 1.6,
    'gravel': 2.0,
    'cobblestone': 2.2,
    'ground': 2.5,
    'dirt': 3.0,
    'unpaved': 2.8,
    'unknown': 4 
}

def surface_weight(u, v, data):
    surface = data.get('surface', 'unknown')
    length = data.get('length', 1)
    factor = surface_weights.get(surface, 4.0)

    peso = length * factor
    # print(f"De {u} a {v}: superficie={surface}, long={length}, factor={factor}, peso={peso}")
    return peso



#### 4.2 Pesos de otras variables 
- force_hospital_weight: con este peso se fuerza a que la ruta pase por un hospital 

In [61]:
def force_hospital_weight(u, v, data):
    surface = data.get('surface', 'unknown')
    length = data.get('length', 1)
    hospitals = data.get('nearby_hospitals', 0)
    
    surface_factor = surface_weights.get(surface, 4.0)
    
    hospital_penalty = 1000 if hospitals == 0 else 1.0
    
    peso = length * surface_factor * hospital_penalty
    return peso

### 5. Enrutamiento 
#### 5.1  Start/End
Con geopy vamos a encontrar la geometría del lugar de inicio y de finalización de la ruta, y asignarlo a el nodo más cercano 

In [68]:
import geopy
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent='myGeocoder')

location_start = locator.geocode('Sede Posgrados Eia, Envigado, Colombia')
location_end=locator.geocode('Universidad EIA, Envigado, Colombia')

start= (location_start.latitude, location_start.longitude)
end= (location_end.latitude, location_end.longitude)

start_node = ox.distance.nearest_nodes(Gsurface, start[1],start[0])
end_node = ox.distance.nearest_nodes(Gsurface, end[1],end[0])

#### 5.2 Busqueda de ruta
Con nx vamos a inicializar las rutas con nuestro nodo de inicio y de finalización 


In [69]:
route1 = nx.shortest_path(G, start_node, end_node, weight=surface_weight)
route_original = nx.shortest_path(Gsurface, start_node, end_node, weight=surface_weight)
route_hospital = nx.shortest_path(Gsurface, start_node, end_node, weight=force_hospital_weight)

Graficas de las rutas 

In [ ]:
ox.plot_graph_route(Gsurface, route1, route_linewidth=6, node_size=0, bgcolor='k',figsize=(20, 20))
ox.plot_graph_route(Gsurface, route_original, route_linewidth=6, node_size=0, bgcolor='w',figsize=(10, 10))
ox.plot_graph_route(Gsurface, route_hospital, route_linewidth=6, node_size=0, bgcolor='w',figsize=(10, 10))

#### 5.3 Funciones complementarias 

1. Grafica de las superficies en envigado 

In [32]:
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import matplotlib.colors as mcolors
# import osmnx as ox

# # Extraer los tipos únicos de superficie para asignar colores
# surfaces = list(gdf_edgesMod['surface'].fillna('unknown').unique())
# surfaces.sort()

# # Crear un mapa de colores personalizado
# color_map = plt.get_cmap("tab20", len(surfaces))
# surface_color_dict = {surf: color_map(i) for i, surf in enumerate(surfaces)}

# # Asignar color a cada arista según su superficie
# edge_colors = gdf_edgesMod['surface'].fillna('unknown').map(surface_color_dict)

# # Dibujar el grafo
# fig, ax = ox.plot_graph(Gsurface,
#                         edge_color=edge_colors.tolist(),
#                         edge_linewidth=3,
#                         node_size=0,
#                         bgcolor='white',
#                         figsize=(30, 30))

# # Mostrar leyenda aparte (opcional)
# legend_patches = [plt.Line2D([0], [0], color=surface_color_dict[surf], lw=3, label=surf) for surf in surfaces]
# plt.figure(figsize=(10, 5))
# plt.legend(handles=legend_patches, loc='center', ncol=4, frameon=False)
# plt.axis('off')
# plt.show()


In [80]:
from datetime import datetime

TRIP_TYPES = ['medical', 'family', 'delivery']

def medical_emergency_weight(u, v, data):
  
    base_cost = data.get('length', 1)
    
    # Superficie
    surface = data.get('surface', 'unknown')
    surface_factors = {
        'asphalt': 1.0, 'concrete': 1.0, 'paved': 1.2,
        'gravel': 3.0, 'unpaved': 4.0, 'unknown': 5.0
    }
    
    surface_factor = surface_factors.get(surface, 5.0)
    
    hospitals = data.get('nearby_hospitals', 0)
    hospital_bonus = 0.3 if hospitals > 0 else 1.0  
    
    # CRÍTICO: Penalizar calles muy estrechas
    lanes = data.get('lanes', '1')
    try:
        num_lanes = int(str(lanes).split(';')[0]) if lanes else 1
        lane_penalty = 1.0 if num_lanes >= 2 else 2.0  
    except:
        lane_penalty = 1.5
    
    # Bonificar carreteras principales
    highway_type = data.get('highway', '')
    highway_bonus = 0.8 if highway_type in ['primary', 'secondary', 'trunk'] else 1.0
    
    return base_cost * surface_factor * hospital_bonus * lane_penalty * highway_bonus
   
   
def family_safe_weight(u, v, data):
    """
    MODO FAMILIAR: Evita zonas industriales, prefiere residenciales
    """
    base_cost = data.get('length', 1)
    
    # Factor superficie (familias prefieren comodidad)
    surface = data.get('surface', 'unknown')
    surface_factors = {
        'asphalt': 1.0, 'concrete': 1.1, 'paved': 1.0,
        'gravel': 2.5, 'unpaved': 3.0, 'unknown': 2.0
    }
    surface_factor = surface_factors.get(surface, 2.0)
    
    # CRÍTICO: Evitar zonas industriales/comerciales pesadas
    highway_type = data.get('highway', '')
    if highway_type in ['industrial', 'trunk']:
        industrial_penalty = 3.0  # Evita fuertemente
    elif highway_type == 'residential':
        industrial_penalty = 0.8  # Prefiere residencial
    else:
        industrial_penalty = 1.0
    
    # Bonificar cercanía a escuelas (zona segura familiar)
    schools = data.get('nearby_schools', 0)
    school_bonus = 0.9 if schools > 0 else 1.0
    
    # Bonificar parques (zona familiar agradable)
    parks = data.get('crosses_park', 0)
    park_bonus = 0.85 if parks > 0 else 1.0
    
    return base_cost * surface_factor * industrial_penalty * school_bonus * park_bonus

   
def heuristic_medical(u, v):
    # Coordenadas de los nodos
    x1, y1 = Gsurface.nodes[u]['x'], Gsurface.nodes[u]['y']
    x2, y2 = Gsurface.nodes[v]['x'], Gsurface.nodes[v]['y']
    
    # Distancia euclidiana
    d = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    
    # Ajuste si hay hospital cerca
    hosp = Gsurface.nodes[u].get('nearby_hospitals', 0)
    return 0.8 * d if hosp > 0 else d


def heuristic_family(u, v):
    x1, y1 = Gsurface.nodes[u]['x'], Gsurface.nodes[u]['y']
    x2, y2 = Gsurface.nodes[v]['x'], Gsurface.nodes[v]['y']
    d = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    park = Gsurface.nodes[u].get('nearby_parks', 0)
    return 0.85 * d if park > 0 else d

MAX_SPEED = 60/3.6  # 60 km/h en m/s

def heuristic_delivery(u, v):
    x1, y1 = Gsurface.nodes[u]['x'], Gsurface.nodes[u]['y']
    x2, y2 = Gsurface.nodes[v]['x'], Gsurface.nodes[v]['y']
    d = ((x2 - x1)**2 + (y2 - y1)**2)**0.5
    return d / MAX_SPEED



In [72]:
def adaptive_emergency_routing(start_node, end_node, trip_type):
    """
    Sistema principal - el usuario debe especificar trip_type
    trip_type: 'medical', 'family', 'delivery'
    """
    if trip_type not in TRIP_TYPES:
        raise ValueError(f"trip_type debe ser uno de: {TRIP_TYPES}")
    
    print(f"Tipo de viaje: {trip_type.upper()}")
    
    # Seleccionar funciones según el tipo
    if trip_type == 'medical':
        weight_func = medical_emergency_weight
        heuristic_func = lambda u, v=end_node: heuristic_medical(u, v)
        print("Modo MÉDICO: Priorizando hospitales y vías amplias")
        
    elif trip_type == 'family':
        weight_func = family_safe_weight
        heuristic_func = lambda u, v=end_node: heuristic_family(u, v)
        print("Modo FAMILIAR: Evitando zonas industriales, prefiriendo seguridad")
        
    elif trip_type == 'delivery':
        weight_func = surface_weight
        heuristic_func = lambda u, v=end_node: heuristic_delivery(u, v)
    
    try:
        route = nx.astar_path(Gsurface, start_node, end_node,
                             heuristic=heuristic_func,
                             weight=weight_func)
        return route
    
    except nx.NetworkXNoPath:
        print(" No se encontró ruta con esta configuración")
        return None

In [73]:
from geopy.geocoders import Nominatim

In [74]:
def interactive_routing(start, end, trip_type):

    locator = Nominatim(user_agent='EnvigadoRoute')
    start_loc = locator.geocode(f"{start}, Envigado, Colombia")
    end_loc = locator.geocode(f"{end}, Envigado, Colombia")
    
    if not start_loc or not end_loc:
        print("Dirección no encontradaaaaaa")
        return
    
    # Obtener nodos
    start_node = ox.distance.nearest_nodes(Gsurface, start_loc.longitude, start_loc.latitude)
    end_node = ox.distance.nearest_nodes(Gsurface, end_loc.longitude, end_loc.latitude)
    
    # Elegir función de peso según tipo de viaje
    if trip_type == 'medical':
        weight_func = medical_emergency_weight
    elif trip_type == 'family':
        weight_func = family_safe_weight
    elif trip_type == 'delivery':
        weight_func = heuristic_delivery
    else:
        print("Tipo de viaje no válido")
        return


    route_adapt = adaptive_emergency_routing(start_node, end_node, trip_type)
    route_orig  = nx.shortest_path(Gsurface, start_node, end_node, weight=weight_func)
    
    if route_adapt:
        print(f"Ruta adaptativa ({trip_type}): {len(route_adapt)} nodos")
        print(f"Ruta normal: {len(route_orig)} nodos")
        colors = {'medical': 'red', 'family': 'green', 'delivery': 'blue'}
        #ox.plot_graph_route(Gsurface, route_adapt, route_color=colors.get(trip_type, 'black'), figsize=(12,12))
        
        def total_weight(route, weight_func):
            total = 0
            for i in range(len(route)-1):
                u, v = route[i], route[i+1]
                data = Gsurface.get_edge_data(u, v)
                if isinstance(data, dict) and 0 in data:
                    data = data[0]
                total += weight_func(u, v, data)
            return total
        
        peso_adapt = total_weight(route_adapt, weight_func)
        peso_orig  = total_weight(route_orig, weight_func)
        
        print(f"Peso total adaptativo: {peso_adapt:.2f}")
        print(f"Peso total original: {peso_orig:.2f}")
        if peso_adapt < peso_orig:
            print("La ruta adaptativa es mejor según el peso")
        else:
            print("La ruta original sigue siendo más eficiente según el peso")
    else:
        print("No se encontró ruta adaptativa")



In [79]:
interactive_routing("Sede Posgrados Eia", "Parque envigado ", trip_type="medical")

Tipo de viaje: MEDICAL
Modo MÉDICO: Priorizando hospitales y vías amplias
Ruta adaptativa (medical): 89 nodos
Ruta normal: 89 nodos
Peso total adaptativo: 3725.62
Peso total original: 3104.67
La ruta original sigue siendo más eficiente según el peso


In [19]:
for u, v, data in Gsurface.edges(data=True):
    print(medical_emergency_weight(u, v, data))


19.310466525532465
73.03537098753127
21.851354198793203
19.310466525532465
86.40921008251871
33.810179917210505
73.03537098753127
21.230025174210905
21.851354198793203
9.701505026329421
21.230025174210905
20.305447090892045
82.73071351477662
19.224148449297356
20.305447090892045
82.73071351477662
19.965283067451722
21.476766974275417
20.711606237407896
50.658630410880995
21.476766974275417
20.711606237407896
21.23766305947113
23.199453584890982
358.05493426400625
103.89589555216713
57.080881102023056
358.05493426400625
27.40313080595338
57.080881102023056
27.62074730715632
27.40313080595338
49.53673061176719
22.694307883730396
24.671136200098758
49.53673061176719
22.694307883730396
8.697221123416258
17.732424974120853
25.320394520464323
15.533086665331595
17.732424974120853
25.320394520464323
27.348644019627745
14.359931335552105
15.374921616491505
15.114840017468396
14.359931335552105
15.374921616491505
19.457480938902307
49.6712140263974
31.144882795031975
28.030664837169684
49.67121

In [21]:
# Elegimos un subconjunto de aristas para no imprimir todo el grafo
edges_to_check = list(Gsurface.edges(data=True))[:10]  # primeras 10 aristas

print("Comparación de pesos:")
for u, v, data in edges_to_check:
    peso_base = data.get('length', 1)  # peso original
    peso_medical = medical_emergency_weight(u, v, data)
    print(f"Arista ({u}, {v}): base={peso_base:.2f}, medical={peso_medical:.2f}")


Comparación de pesos:
Arista (309313450, 414940715): base=24.14, medical=19.31
Arista (309313450, 440765240): base=91.29, medical=73.04
Arista (414940715, 309313451): base=27.31, medical=21.85
Arista (414940715, 309313450): base=24.14, medical=19.31
Arista (440765240, 440765238): base=43.20, medical=86.41
Arista (440765240, 5324852837): base=42.26, medical=33.81
Arista (440765240, 309313450): base=91.29, medical=73.04
Arista (309313451, 414940727): base=26.54, medical=21.23
Arista (309313451, 414940715): base=27.31, medical=21.85
Arista (414940727, 9400446431): base=12.13, medical=9.70
